<a href="https://colab.research.google.com/github/yecatstevir/teambrainiac/blob/main/source/group_svm/Statistical_tests_decisions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Two Sample T-test Between Young Adult and Adolescent using decision scores of the significant voxels decided by the SVM. 

- Target Whole Brain
- Between Groups
- Between WB and Submasks and WB and ROIs


In [1]:
from google.colab import drive, files
drive.mount('/content/gdrive')#, force_remount = True)

# Clone the entire repo.
!git clone -l -s https://github.com/yecatstevir/teambrainiac.git
# Change directory into cloned repo
%cd teambrainiac/source/group_svm
!ls
"""uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))"""

Mounted at /content/gdrive
Cloning into 'teambrainiac'...
remote: Enumerating objects: 2308, done.
remote: Counting objects: 100% (183/183), done.
remote: Compressing objects: 100% (169/169), done.
remote: Total 2308 (delta 108), reused 32 (delta 13), pack-reused 2125
Receiving objects: 100% (2308/2308), 153.41 MiB | 11.67 MiB/s, done.
Resolving deltas: 100% (1499/1499), done.
/content/teambrainiac/source/group_svm
access_data.py		    __init__.py
Adolescent_Group_SVM.ipynb  process.py
analysis.py		    Statistical_tests_Group.ipynb
cross_validation.py	    Timeseries_Cross_Validation.ipynb
data			    train.py
Explore_data.ipynb	    visualize.py
Group_metrics.ipynb	    Young_Adult_Group_SVM.ipynb
images


'uploaded = files.upload()\nfor fn in uploaded.keys():\n  print(\'User uploaded file "{name}" with length {length} bytes\'.format(\n      name=fn, length=len(uploaded[fn])))'

In [2]:
# Import libraries
!pip install boto3 nilearn
from process import *
from access_data import *
from scipy import stats
import pandas as pd
import numpy as np

     |████████████████████████████████| 132 kB 5.2 MB/s 
     |████████████████████████████████| 9.6 MB 25.1 MB/s 
     |████████████████████████████████| 79 kB 9.4 MB/s 
     |████████████████████████████████| 8.7 MB 48.6 MB/s 
     |████████████████████████████████| 138 kB 59.5 MB/s 
     |████████████████████████████████| 38.1 MB 1.2 MB/s 
     |████████████████████████████████| 127 kB 60.7 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: scipy
    Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
albumentations 0.1.12 r

> Load decision scores for ROIs

In [47]:
import pandas as pd
import numpy as np
from collections import defaultdict
adpfc = open_pickle('/content/gdrive/MyDrive/AD_detrend_mPFC_decisions.pkl')
adnacc = open_pickle('/content/gdrive/MyDrive/AD_detrend_nacc_aal_decisions.pkl')
adacc = open_pickle('/content/gdrive/MyDrive/AD_detrend_acc_aal_decisions.pkl')
adai = open_pickle('/content/gdrive/MyDrive/AD_detrend_anterior_insula_aal_decisions.pkl')

ad_mpfc = adpfc['decision']
ad_nacc = adnacc['decision']
ad_acc = adacc['decision']
ad_ai = adai['decision']

yapfc = open_pickle('/content/gdrive/MyDrive/YA_detrend_mPFC_decisions.pkl')
yanacc = open_pickle('/content/gdrive/MyDrive/YA_detrend_nacc_aal_decisions.pkl')
yaacc = open_pickle('/content/gdrive/MyDrive/YA_detrend_acc_aal_decisions.pkl')
yaai = open_pickle('/content/gdrive/MyDrive/YA_detrend_anterior_insula_aal_decisions.pkl')

ya_mpfc = yapfc['decision']
ya_nacc = yanacc['decision']
ya_acc = yaacc['decision']
ya_ai = yaai['decision']

Decision scores between YA and AD : WB, SUB

Decision scores between YA and AD : WB, ROI

Decision scores between YA and AD : ROIs

Decision scores between YA and AD : SUB

Decision scores between YA and AD : WB

Decision scores between AD : WB, SUB

Decision scores between AD : WB, ROI

Decision scores between YA : WB, SUB

Decision scores between YA : WB, ROI

Decision scores between AD : SUB

Decision scores between YA : SUB

Decision scores between AD : ROI

Decision scores between YA : ROI

### Whole Brain Analysis Between Adolescent and Young Adult 2 Sample t-test

> Load Young Adult Whole Brain Information

In [3]:
ya_sub = defaultdict(list)
# Load model
#open path dictionary file to get subject ids
dict_path = "data/data_path_dictionary.pkl"
data_path_dict = open_pickle(dict_path)

mask_types = ['mask', 'masksubACC', 'masksubAI', 'masksubNAcc', 'masksubmPFC']
data_type = "YA_detrend"
runs_train = [1, 2]
runs_id = [i + 1 for i in runs_train]
m_path_ind = 0 #get sub_mask data in mask_data key
for mask_type in mask_types:
  metric_ya = access_load_data(f'metrics/group_svm/{mask_type}/{data_type}_{runs_id}_{mask_type}_metrics.pkl', False)
  ya_sub[f'{mask_type}_dscore'].append(metric_ya['test_dfunc'][0])

In [8]:
ya_sub['mask_dscore'][0]

(672,)

In [51]:
ya_wb = ya_sub['mask_dscore'][0]
ya_subacc = ya_sub['masksubACC_dscore'][0]
ya_subai = ya_sub['masksubAI_dscore'][0]
ya_subpfc = ya_sub['masksubmPFC_dscore'][0]
ya_subncaa = ya_sub['masksubNAcc_dscore'][0]

> Load Adolescent Whole Brain Information

In [4]:
# Load model
ad_sub = defaultdict(list)
mask_types = ['mask', 'masksubACC', 'masksubAI', 'masksubNAcc', 'masksubmPFC']
data_type = "AD_detrend"
runs_train = [1, 2]
runs_id = [i + 1 for i in runs_train]
m_path_ind = 0 #get sub_mask data in mask_data key
for mask_type in mask_types:
  metric_ya = access_load_data(f'metrics/group_svm/{mask_type}/{data_type}_{runs_id}_{mask_type}_metrics.pkl', False)
  ad_sub[f'{mask_type}_dscore'].append(metric_ya['test_dfunc'][0])

In [50]:
ad_wb = ad_sub['mask_dscore'][0]
ad_subacc = ad_sub['masksubACC_dscore'][0]
ad_subai = ad_sub['masksubAI_dscore'][0]
ad_subpfc = ad_sub['masksubmPFC_dscore'][0]
ad_subncaa = ad_sub['masksubNAcc_dscore'][0]

> Whole Brain Summary Statistics

In [12]:
dictionary_values1 = {'YA': ya_wb}
dictionary_values2 = {'AD':ad_wb}
summary_df1 = pd.DataFrame(dictionary_values1)
print(summary_df1.describe())
summary_df2 = pd.DataFrame(dictionary_values2)
print(summary_df2.describe())

               YA
count  672.000000
mean     0.011738
std      0.434251
min     -0.946131
25%     -0.306588
50%      0.006389
75%      0.339271
max      1.361713
                AD
count  1176.000000
mean      0.035384
std       0.232994
min      -0.629681
25%      -0.115465
50%       0.028632
75%       0.192001
max       0.705184


In [13]:
t_value, p_value = stats.ttest_ind(ya_wb, ad_wb)

print(f'Test statistic is {t_value}')

print(f'p-value for two tailed test is {p_value}')

alpha = 0.05

if p_value <= alpha:
  print(f'Conclusion: Since p-value {p_value} < alpha {alpha}, We reject the null hypothesis H0. \
  \nSo we conclude that at the whole brain level, Young Adults and Adolescents do not regulate the same. \
  \ni.e., μ1 = μ2 at {alpha} level of significance.')

else:

  print(f'Conclusion Since p-value{p_value} > alpha {alpha},We do not reject the null hypothesis H0.')


Test statistic is -1.522848501123466
p-value for two tailed test is 0.1279678891770675
Conclusion Since p-value0.1279678891770675 > alpha 0.05,We do not reject the null hypothesis H0.


### SubACC
- SubACC between YA, AD
- WB SubACC YA
- WB SubACC AD



In [14]:
dictionary_values1 = {'YA': ya_subacc}
dictionary_values2 = {'AD':ad_subacc}
summary_df1 = pd.DataFrame(dictionary_values1)
print(summary_df1.describe())
summary_df2 = pd.DataFrame(dictionary_values2)
print(summary_df2.describe())

               YA
count  672.000000
mean     0.011863
std      0.429416
min     -0.937269
25%     -0.309286
50%      0.015745
75%      0.330064
max      1.354795
                AD
count  1176.000000
mean      0.043694
std       0.255499
min      -0.709318
25%      -0.126833
50%       0.038064
75%       0.217944
max       0.753787


In [16]:
t_value, p_value = stats.ttest_ind(ya_subacc, ad_subacc)

print(f'Test statistic is {t_value}')

print(f'p-value for two tailed test is {p_value}')

alpha = 0.05

if p_value <= alpha:
  print(f'Conclusion: Since p-value {p_value} < alpha {alpha}, We reject the null hypothesis H0. \
  \nSo we conclude that when removing the ACC, Young Adults and Adolescents do not regulate the same. \
  \ni.e., μ1 = μ2 at {alpha} level of significance.')

else:

  print(f'Conclusion Since p-value{p_value} > alpha {alpha},We do not reject the null hypothesis H0.')

Test statistic is -1.997647484833968
p-value for two tailed test is 0.04590140664863833
Conclusion: Since p-value 0.04590140664863833 < alpha 0.05, We reject the null hypothesis H0.   
So we conclude that when removing the ACC, Young Adults and Adolescents do not regulate the same.   
i.e., μ1 = μ2 at 0.05 level of significance.


In [17]:
t_value, p_value = stats.ttest_ind(ya_wb, ya_subacc)

print(f'Test statistic is {t_value}')

print(f'p-value for two tailed test is {p_value}')

alpha = 0.05

if p_value <= alpha:
  print(f'Conclusion: Since p-value {p_value} < alpha {alpha}, We reject the null hypothesis H0. \
  \nSo we conclude that when removing the ACC, Young Adults do not regulate the same. \
  \ni.e., μ1 = μ2 at {alpha} level of significance.')

else:

  print(f'Conclusion Since p-value{p_value} > alpha {alpha},We do not reject the null hypothesis H0.')

Test statistic is -0.005299001913581159
p-value for two tailed test is 0.9957728155385006
Conclusion Since p-value0.9957728155385006 > alpha 0.05,We do not reject the null hypothesis H0.


In [18]:
t_value, p_value = stats.ttest_ind(ad_wb, ad_subacc)

print(f'Test statistic is {t_value}')

print(f'p-value for two tailed test is {p_value}')

alpha = 0.05

if p_value <= alpha:
  print(f'Conclusion: Since p-value {p_value} < alpha {alpha}, We reject the null hypothesis H0. \
  \nSo we conclude that when removing the ACC, Adolescents do not regulate the same. \
  \ni.e., μ1 = μ2 at {alpha} level of significance.')

else:

  print(f'Conclusion Since p-value{p_value} > alpha {alpha},We do not reject the null hypothesis H0.')

Test statistic is -0.8242036737394411
p-value for two tailed test is 0.40990748257795406
Conclusion Since p-value0.40990748257795406 > alpha 0.05,We do not reject the null hypothesis H0.


### SubAI
- SubAI between YA, AD
- WB SubAI YA
- WB SubAI AD

In [22]:
dictionary_values1 = {'YA': ya_subai}
dictionary_values2 = {'AD':ad_subai}
summary_df1 = pd.DataFrame(dictionary_values1)
print(summary_df1.describe())
summary_df2 = pd.DataFrame(dictionary_values2)
print(summary_df2.describe())

               YA
count  672.000000
mean     0.010594
std      0.432705
min     -0.957052
25%     -0.307282
50%      0.014027
75%      0.338142
max      1.347922
                AD
count  1176.000000
mean      0.037443
std       0.243762
min      -0.679624
25%      -0.122542
50%       0.034368
75%       0.199172
max       0.734372


In [23]:
t_value, p_value = stats.ttest_ind(ya_subai, ad_subai)

print(f'Test statistic is {t_value}')

print(f'p-value for two tailed test is {p_value}')

alpha = 0.05

if p_value <= alpha:
  print(f'Conclusion: Since p-value {p_value} < alpha {alpha}, We reject the null hypothesis H0. \
  \nSo we conclude that when removing the AI, Young Adults and Adolescents do not regulate the same. \
  \ni.e., μ1 = μ2 at {alpha} level of significance.')

else:

  print(f'Conclusion Since p-value{p_value} > alpha {alpha},We do not reject the null hypothesis H0.')

Test statistic is -1.7063401013362085
p-value for two tailed test is 0.08811297186377545
Conclusion Since p-value0.08811297186377545 > alpha 0.05,We do not reject the null hypothesis H0.


In [24]:
t_value, p_value = stats.ttest_ind(ya_wb, ya_subai)

print(f'Test statistic is {t_value}')

print(f'p-value for two tailed test is {p_value}')

alpha = 0.05

if p_value <= alpha:
  print(f'Conclusion: Since p-value {p_value} < alpha {alpha}, We reject the null hypothesis H0. \
  \nSo we conclude that when removing the AI, Young Adults do not regulate the same. \
  \ni.e., μ1 = μ2 at {alpha} level of significance.')

else:

  print(f'Conclusion Since p-value{p_value} > alpha {alpha},We do not reject the null hypothesis H0.')

Test statistic is 0.048395708581289824
p-value for two tailed test is 0.9614080804953828
Conclusion Since p-value0.9614080804953828 > alpha 0.05,We do not reject the null hypothesis H0.


In [25]:
t_value, p_value = stats.ttest_ind(ad_wb, ad_subai)

print(f'Test statistic is {t_value}')

print(f'p-value for two tailed test is {p_value}')

alpha = 0.05

if p_value <= alpha:
  print(f'Conclusion: Since p-value {p_value} < alpha {alpha}, We reject the null hypothesis H0. \
  \nSo we conclude that when removing the AI, Adolescents do not regulate the same. \
  \ni.e., μ1 = μ2 at {alpha} level of significance.')

else:

  print(f'Conclusion Since p-value{p_value} > alpha {alpha},We do not reject the null hypothesis H0.')

Test statistic is -0.20944300739866234
p-value for two tailed test is 0.8341205761707682
Conclusion Since p-value0.8341205761707682 > alpha 0.05,We do not reject the null hypothesis H0.


### SubmPFC
- SubmPFC between YA, AD
- WB SubmPFC YA
- WB SubmPFC AD

In [27]:
dictionary_values1 = {'YA': ya_subpfc}
dictionary_values2 = {'AD':ad_subpfc}
summary_df1 = pd.DataFrame(dictionary_values1)
print(summary_df1.describe())
summary_df2 = pd.DataFrame(dictionary_values2)
print(summary_df2.describe())

               YA
count  672.000000
mean     0.009688
std      0.442777
min     -0.971446
25%     -0.316242
50%      0.015702
75%      0.350304
max      1.354417
                AD
count  1176.000000
mean      0.034377
std       0.255183
min      -0.718930
25%      -0.139089
50%       0.032431
75%       0.207754
max       0.827596


In [31]:
t_value, p_value = stats.ttest_ind(ya_subpfc, ad_subpfc)

print(f'Test statistic is {t_value}')

print(f'p-value for two tailed test is {p_value}')

alpha = 0.05

if p_value <= alpha:
  print(f'Conclusion: Since p-value {p_value} < alpha {alpha}, We reject the null hypothesis H0. \
  \nSo we conclude that when removing the mPFC, Young Adults and Adolescents do not regulate the same. \
  \ni.e., μ1 = μ2 at {alpha} level of significance.')

else:

  print(f'Conclusion Since p-value{p_value} > alpha {alpha},We do not reject the null hypothesis H0.')

Test statistic is -1.520726421172203
p-value for two tailed test is 0.12849976272304303
Conclusion Since p-value0.12849976272304303 > alpha 0.05,We do not reject the null hypothesis H0.


In [30]:
t_value, p_value = stats.ttest_ind(ya_wb, ya_subpfc)

print(f'Test statistic is {t_value}')

print(f'p-value for two tailed test is {p_value}')

alpha = 0.05

if p_value <= alpha:
  print(f'Conclusion: Since p-value {p_value} < alpha {alpha}, We reject the null hypothesis H0. \
  \nSo we conclude that when removing the mPFC, Young Adults do not regulate the same. \
  \ni.e., μ1 = μ2 at {alpha} level of significance.')

else:

  print(f'Conclusion Since p-value{p_value} > alpha {alpha},We do not reject the null hypothesis H0.')

Test statistic is 0.08570068637576371
p-value for two tailed test is 0.9317171401238309
Conclusion Since p-value0.9317171401238309 > alpha 0.05,We do not reject the null hypothesis H0.


In [32]:
t_value, p_value = stats.ttest_ind(ad_wb, ad_subpfc)

print(f'Test statistic is {t_value}')

print(f'p-value for two tailed test is {p_value}')

alpha = 0.05

if p_value <= alpha:
  print(f'Conclusion: Since p-value {p_value} < alpha {alpha}, We reject the null hypothesis H0. \
  \nSo we conclude that when removing the mPFC, Adolescents do not regulate the same. \
  \ni.e., μ1 = μ2 at {alpha} level of significance.')

else:

  print(f'Conclusion Since p-value{p_value} > alpha {alpha},We do not reject the null hypothesis H0.')

Test statistic is 0.09993364478933432
p-value for two tailed test is 0.9204055294449156
Conclusion Since p-value0.9204055294449156 > alpha 0.05,We do not reject the null hypothesis H0.


### SubNAcc
- SubNAcc between YA, AD
- WB SubNAcc YA
- WB SubNAcc AD

In [52]:
dictionary_values1 = {'YA': ya_subncaa}
dictionary_values2 = {'AD':ad_subncaa}
summary_df1 = pd.DataFrame(dictionary_values1)
print(summary_df1.describe())
summary_df2 = pd.DataFrame(dictionary_values2)
print(summary_df2.describe())

               YA
count  672.000000
mean     0.011158
std      0.435743
min     -0.968558
25%     -0.316110
50%      0.010255
75%      0.337425
max      1.342921
                AD
count  1176.000000
mean      0.040221
std       0.256145
min      -0.693427
25%      -0.132655
50%       0.032157
75%       0.212106
max       0.781820


In [35]:
t_value, p_value = stats.ttest_ind(ya_subncaa, ad_subncaa)

print(f'Test statistic is {t_value}')

print(f'p-value for two tailed test is {p_value}')

alpha = 0.05

if p_value <= alpha:
  print(f'Conclusion: Since p-value {p_value} < alpha {alpha}, We reject the null hypothesis H0. \
  \nSo we conclude that when removing the Nucleus Accumbens, Young Adults and Adolescents do not regulate the same. \
  \ni.e., μ1 = μ2 at {alpha} level of significance.')

else:

  print(f'Conclusion Since p-value{p_value} > alpha {alpha},We do not reject the null hypothesis H0.')

Test statistic is -1.8056955785421944
p-value for two tailed test is 0.0711287448526961
Conclusion Since p-value0.0711287448526961 > alpha 0.05,We do not reject the null hypothesis H0.


In [36]:
t_value, p_value = stats.ttest_ind(ya_wb, ya_subncaa)

print(f'Test statistic is {t_value}')

print(f'p-value for two tailed test is {p_value}')

alpha = 0.05

if p_value <= alpha:
  print(f'Conclusion: Since p-value {p_value} < alpha {alpha}, We reject the null hypothesis H0. \
  \nSo we conclude that when removing the Nucleus Accumbens, Young Adults do not regulate the same. \
  \ni.e., μ1 = μ2 at {alpha} level of significance.')

else:

  print(f'Conclusion Since p-value{p_value} > alpha {alpha},We do not reject the null hypothesis H0.')

Test statistic is 0.024430050690201446
p-value for two tailed test is 0.9805132104491241
Conclusion Since p-value0.9805132104491241 > alpha 0.05,We do not reject the null hypothesis H0.


In [37]:
t_value, p_value = stats.ttest_ind(ad_wb, ad_subncaa)

print(f'Test statistic is {t_value}')

print(f'p-value for two tailed test is {p_value}')

alpha = 0.05

if p_value <= alpha:
  print(f'Conclusion: Since p-value {p_value} < alpha {alpha}, We reject the null hypothesis H0. \
  \nSo we conclude that when removing the Nucleus Accumbens,  Adolescents do not regulate the same. \
  \ni.e., μ1 = μ2 at {alpha} level of significance.')

else:

  print(f'Conclusion Since p-value{p_value} > alpha {alpha},We do not reject the null hypothesis H0.')

Test statistic is -0.47909089052296444
p-value for two tailed test is 0.6319185457862074
Conclusion Since p-value0.6319185457862074 > alpha 0.05,We do not reject the null hypothesis H0.


###ACC
- ACC between YA, AD
- WB ACC YA
- WB ACC AD

In [57]:
dictionary_values1 = {'YA': ya_acc}
dictionary_values2 = {'AD':ad_acc}
summary_df1 = pd.DataFrame(dictionary_values1)
print(summary_df1.describe())
summary_df2 = pd.DataFrame(dictionary_values2)
print(summary_df2.describe())

               YA
count  672.000000
mean    -0.003285
std      0.426422
min     -1.148944
25%     -0.294057
50%      0.007557
75%      0.306150
max      1.360153
                AD
count  1176.000000
mean      0.011973
std       0.235729
min      -0.836195
25%      -0.136994
50%       0.007399
75%       0.168923
max       0.804348


In [58]:
t_value, p_value = stats.ttest_ind(ya_acc, ad_acc)

print(f'Test statistic is {t_value}')

print(f'p-value for two tailed test is {p_value}')

alpha = 0.05

if p_value <= alpha:
  print(f'Conclusion: Since p-value {p_value} < alpha {alpha}, We reject the null hypothesis H0. \
  \nSo we conclude that when looking at the ACC, Young Adults and Adolescents do not regulate the same. \
  \ni.e., μ1 = μ2 at {alpha} level of significance.')

else:

  print(f'Conclusion Since p-value{p_value} > alpha {alpha},We do not reject the null hypothesis H0.')

Test statistic is -0.9904983490002937
p-value for two tailed test is 0.32206040536048985
Conclusion Since p-value0.32206040536048985 > alpha 0.05,We do not reject the null hypothesis H0.


In [59]:
t_value, p_value = stats.ttest_ind(ya_wb, ya_acc)

print(f'Test statistic is {t_value}')

print(f'p-value for two tailed test is {p_value}')

alpha = 0.05

if p_value <= alpha:
  print(f'Conclusion: Since p-value {p_value} < alpha {alpha}, We reject the null hypothesis H0. \
  \nSo we conclude that when looking at the ACC, Young Adults do not regulate the same. \
  \ni.e., μ1 = μ2 at {alpha} level of significance.')

else:

  print(f'Conclusion Since p-value{p_value} > alpha {alpha},We do not reject the null hypothesis H0.')

Test statistic is 0.639877101966053
p-value for two tailed test is 0.5223617221705159
Conclusion Since p-value0.5223617221705159 > alpha 0.05,We do not reject the null hypothesis H0.


In [60]:
t_value, p_value = stats.ttest_ind(ad_wb, ad_acc)

print(f'Test statistic is {t_value}')

print(f'p-value for two tailed test is {p_value}')

alpha = 0.05

if p_value <= alpha:
  print(f'Conclusion: Since p-value {p_value} < alpha {alpha}, We reject the null hypothesis H0. \
  \nSo we conclude that when looking at the ACC, Adolescents do not regulate the same. \
  \ni.e., μ1 = μ2 at {alpha} level of significance.')

else:

  print(f'Conclusion Since p-value{p_value} > alpha {alpha},We do not reject the null hypothesis H0.')

Test statistic is 2.4222297318468575
p-value for two tailed test is 0.015500780796081328
Conclusion: Since p-value 0.015500780796081328 < alpha 0.05, We reject the null hypothesis H0.   
So we conclude that when looking at the ACC, Adolescents do not regulate the same.   
i.e., μ1 = μ2 at 0.05 level of significance.


###AI
- AI between YA, AD
- WB AI YA
- WB AI AD

In [61]:
dictionary_values1 = {'YA': ya_ai}
dictionary_values2 = {'AD':ad_ai}
summary_df1 = pd.DataFrame(dictionary_values1)
print(summary_df1.describe())
summary_df2 = pd.DataFrame(dictionary_values2)
print(summary_df2.describe())

               YA
count  672.000000
mean    -0.032144
std      0.361382
min     -1.370008
25%     -0.266359
50%     -0.036388
75%      0.205521
max      0.994335
                AD
count  1176.000000
mean     -0.070185
std       0.328094
min      -1.082912
25%      -0.288165
50%      -0.076824
75%       0.136764
max       1.006725


In [62]:
t_value, p_value = stats.ttest_ind(ya_ai, ad_ai)

print(f'Test statistic is {t_value}')

print(f'p-value for two tailed test is {p_value}')

alpha = 0.05

if p_value <= alpha:
  print(f'Conclusion: Since p-value {p_value} < alpha {alpha}, We reject the null hypothesis H0. \
  \nSo we conclude that when looking at the AI, Young Adults and Adolescents do not regulate the same. \
  \ni.e., μ1 = μ2 at {alpha} level of significance.')

else:

  print(f'Conclusion Since p-value{p_value} > alpha {alpha},We do not reject the null hypothesis H0.')

Test statistic is 2.3098256700141833
p-value for two tailed test is 0.0210076575061983
Conclusion: Since p-value 0.0210076575061983 < alpha 0.05, We reject the null hypothesis H0.   
So we conclude that when looking at the AI, Young Adults and Adolescents do not regulate the same.   
i.e., μ1 = μ2 at 0.05 level of significance.


In [65]:
t_value, p_value = stats.ttest_ind(ya_wb,ya_ai)

print(f'Test statistic is {t_value}')

print(f'p-value for two tailed test is {p_value}')

alpha = 0.05

if p_value <= alpha:
  print(f'Conclusion: Since p-value {p_value} < alpha {alpha}, We reject the null hypothesis H0. \
  \nSo we conclude that when looking at the AI, Young Adults do not regulate the same. \
  \ni.e., μ1 = μ2 at {alpha} level of significance.')

else:

  print(f'Conclusion Since p-value{p_value} > alpha {alpha},We do not reject the null hypothesis H0.')

Test statistic is 2.0135548193680517
p-value for two tailed test is 0.04425560089777816
Conclusion: Since p-value 0.04425560089777816 < alpha 0.05, We reject the null hypothesis H0.   
So we conclude that when looking at the AI, Young Adults do not regulate the same.   
i.e., μ1 = μ2 at 0.05 level of significance.


In [66]:
t_value, p_value = stats.ttest_ind(ad_wb, ad_ai)

print(f'Test statistic is {t_value}')

print(f'p-value for two tailed test is {p_value}')

alpha = 0.05

if p_value <= alpha:
  print(f'Conclusion: Since p-value {p_value} < alpha {alpha}, We reject the null hypothesis H0. \
  \nSo we conclude that when looking at the AI,  Adolescents do not regulate the same. \
  \ni.e., μ1 = μ2 at {alpha} level of significance.')

else:

  print(f'Conclusion Since p-value{p_value} > alpha {alpha},We do not reject the null hypothesis H0.')

Test statistic is 8.99646072972302
p-value for two tailed test is 4.685453837941732e-19
Conclusion: Since p-value 4.685453837941732e-19 < alpha 0.05, We reject the null hypothesis H0.   
So we conclude that when looking at the AI,  Adolescents do not regulate the same.   
i.e., μ1 = μ2 at 0.05 level of significance.


###mPFC
- mPFC between YA, AD
- WB mPFC YA
- WB mPFC AD

In [67]:
dictionary_values1 = {'YA': ya_mpfc}
dictionary_values2 = {'AD':ad_mpfc}
summary_df1 = pd.DataFrame(dictionary_values1)
print(summary_df1.describe())
summary_df2 = pd.DataFrame(dictionary_values2)
print(summary_df2.describe())

               YA
count  672.000000
mean     0.037698
std      0.511252
min     -1.536291
25%     -0.282303
50%      0.052453
75%      0.364933
max      1.486680
                AD
count  1176.000000
mean     -0.033614
std       0.398963
min      -1.398510
25%      -0.303652
50%      -0.016756
75%       0.242795
max       1.139247


In [68]:
t_value, p_value = stats.ttest_ind(ya_mpfc, ad_mpfc)

print(f'Test statistic is {t_value}')

print(f'p-value for two tailed test is {p_value}')

alpha = 0.05

if p_value <= alpha:
  print(f'Conclusion: Since p-value {p_value} < alpha {alpha}, We reject the null hypothesis H0. \
  \nSo we conclude that when looking at the mPFC, Young Adults and Adolescents do not regulate the same. \
  \ni.e., μ1 = μ2 at {alpha} level of significance.')

else:

  print(f'Conclusion Since p-value{p_value} > alpha {alpha},We do not reject the null hypothesis H0.')

Test statistic is 3.328250878597839
p-value for two tailed test is 0.0008911102551991956
Conclusion: Since p-value 0.0008911102551991956 < alpha 0.05, We reject the null hypothesis H0.   
So we conclude that when looking at the mPFC, Young Adults and Adolescents do not regulate the same.   
i.e., μ1 = μ2 at 0.05 level of significance.


In [69]:
t_value, p_value = stats.ttest_ind(ya_wb, ya_mpfc)

print(f'Test statistic is {t_value}')

print(f'p-value for two tailed test is {p_value}')

alpha = 0.05

if p_value <= alpha:
  print(f'Conclusion: Since p-value {p_value} < alpha {alpha}, We reject the null hypothesis H0. \
  \nSo we conclude that when looking at the mPFC, Young Adults do not regulate the same. \
  \ni.e., μ1 = μ2 at {alpha} level of significance.')

else:

  print(f'Conclusion Since p-value{p_value} > alpha {alpha},We do not reject the null hypothesis H0.')

Test statistic is -1.0032239616861343
p-value for two tailed test is 0.3159336668344514
Conclusion Since p-value0.3159336668344514 > alpha 0.05,We do not reject the null hypothesis H0.


In [70]:
t_value, p_value = stats.ttest_ind(ad_wb, ad_mpfc)

print(f'Test statistic is {t_value}')

print(f'p-value for two tailed test is {p_value}')

alpha = 0.05

if p_value <= alpha:
  print(f'Conclusion: Since p-value {p_value} < alpha {alpha}, We reject the null hypothesis H0. \
  \nSo we conclude that when looking at the mPFC, Adolescents do not regulate the same. \
  \ni.e., μ1 = μ2 at {alpha} level of significance.')

else:

  print(f'Conclusion Since p-value{p_value} > alpha {alpha},We do not reject the null hypothesis H0.')

Test statistic is 5.1213497890464055
p-value for two tailed test is 3.28001899097621e-07
Conclusion: Since p-value 3.28001899097621e-07 < alpha 0.05, We reject the null hypothesis H0.   
So we conclude that when looking at the ACC, Adolescents do not regulate the same.   
i.e., μ1 = μ2 at 0.05 level of significance.


###NCaa
- NCaa between YA, AD
- WB NCaa YA
- WB NCaa AD

In [71]:
dictionary_values1 = {'YA': ya_ncaa}
dictionary_values2 = {'AD':ad_ncaa}
summary_df1 = pd.DataFrame(dictionary_values1)
print(summary_df1.describe())
summary_df2 = pd.DataFrame(dictionary_values2)
print(summary_df2.describe())

               YA
count  672.000000
mean     0.011158
std      0.435743
min     -0.968558
25%     -0.316110
50%      0.010255
75%      0.337425
max      1.342921
                AD
count  1176.000000
mean      0.040221
std       0.256145
min      -0.693427
25%      -0.132655
50%       0.032157
75%       0.212106
max       0.781820


In [72]:
t_value, p_value = stats.ttest_ind(ya_ncaa, ad_ncaa)

print(f'Test statistic is {t_value}')

print(f'p-value for two tailed test is {p_value}')

alpha = 0.05

if p_value <= alpha:
  print(f'Conclusion: Since p-value {p_value} < alpha {alpha}, We reject the null hypothesis H0. \
  \nSo we conclude that when looking at the Nucleus Accumbens, Young Adults and Adolescents do not regulate the same. \
  \ni.e., μ1 = μ2 at {alpha} level of significance.')

else:

  print(f'Conclusion Since p-value{p_value} > alpha {alpha},We do not reject the null hypothesis H0.')

Test statistic is -1.8056955785421944
p-value for two tailed test is 0.0711287448526961
Conclusion Since p-value0.0711287448526961 > alpha 0.05,We do not reject the null hypothesis H0.


In [73]:
t_value, p_value = stats.ttest_ind(ya_wb, ya_ncaa)

print(f'Test statistic is {t_value}')

print(f'p-value for two tailed test is {p_value}')

alpha = 0.05

if p_value <= alpha:
  print(f'Conclusion: Since p-value {p_value} < alpha {alpha}, We reject the null hypothesis H0. \
  \nSo we conclude that when looking at the Nucleus Accumbens, Young Adults  do not regulate the same. \
  \ni.e., μ1 = μ2 at {alpha} level of significance.')

else:

  print(f'Conclusion Since p-value{p_value} > alpha {alpha},We do not reject the null hypothesis H0.')

Test statistic is 0.024430050690201446
p-value for two tailed test is 0.9805132104491241
Conclusion Since p-value0.9805132104491241 > alpha 0.05,We do not reject the null hypothesis H0.


In [74]:
t_value, p_value = stats.ttest_ind(ad_wb, ad_ncaa)

print(f'Test statistic is {t_value}')

print(f'p-value for two tailed test is {p_value}')

alpha = 0.05

if p_value <= alpha:
  print(f'Conclusion: Since p-value {p_value} < alpha {alpha}, We reject the null hypothesis H0. \
  \nSo we conclude that when looking at  the Nucleus Accumbens, Adolescents do not regulate the same. \
  \ni.e., μ1 = μ2 at {alpha} level of significance.')

else:

  print(f'Conclusion Since p-value{p_value} > alpha {alpha},We do not reject the null hypothesis H0.')

Test statistic is -0.47909089052296444
p-value for two tailed test is 0.6319185457862074
Conclusion Since p-value0.6319185457862074 > alpha 0.05,We do not reject the null hypothesis H0.


> Create Tables

In [9]:
import pandas as pd
#!pip install -U kaleido
dictionary = {
              "yf": {"Region": "Medial Prefrontal Cortex decision score ", "Test Statistic":3.32825, "p-value":0.000891, "alpha":0.05,"Hypothesis": "Reject Null H0"},
              "hgfj": {"Region":"Anterior Insula (Right side)","Test Statistic":2.309826, "p-value": 0.021008, "alpha":0.05, "Hypothesis": "Reject Null H0"},
              "fdh": {"Region":"Whole Brain without the Anterior Cingulate Cortex","Test Statistic":-1.997647, "p-value":0.045901, "alpha":0.05, "Hypothesis": "Reject Null H0"},
              }

df = pd.DataFrame(dictionary)
dff= df.T

import plotly.graph_objects as go
import pandas as pd
import plotly.io as pio

fig = go.Figure(data=[go.Table(
    header=dict(values=dff.columns,
                fill_color='#96c6e2',
                align='center'),
    cells=dict(values=[dff['Region'],dff['Test Statistic'], dff['p-value'], dff['alpha'],dff['Hypothesis']],
               fill_color='#eaeaf2',
               align='left'))
])

fig.update_layout(
    height=650,
    width = 700,
    title_text="Two Tailed T-Test Between Young Adult and Adolescent Decision Scores",
    title_x=0.5
)
import os

if not os.path.exists("data"):
    os.mkdir("data")
fig.write_image("data/dscore_grou_ttst.png", scale = 2)
fig.show()

In [12]:
import pandas as pd
#!pip install -U kaleido
dictionary = {
              "yf": {"Region": "Adolescent Whole Brain and Medial Prefrontal Cortex decision score ", "Test Statistic":5.12135, "p-value":'3.280019e-07', "alpha":0.05,"Hypothesis": "Reject Null H0"},
              "hgfj":{"Region":"Adolecent Whole Brain and Anterior Cingulate Cortex", "Test Statistic": 2.42223, "p-value": 0.015501, "alpha":0.05, "Hypothesis": "Reject Null H0"},
              "ftfh": {"Region":"Adolescent Whole Brain and Anterior Insula (Right side)", "Test Statistic":8.99646, "p-value":'4.68545e-19', "alpha":0.05, "Hypothesis": "Reject Null H0"},
              "fdh": {"Region":"Young Adult Whole Brain and Anterior Insula (Ride side)", "Test Statistic":2.01355, "p-value":0.0442556, "alpha":0.05, "Hypothesis": "Reject Null H0"}
              }

df = pd.DataFrame(dictionary)
dff= df.T

import plotly.graph_objects as go
import pandas as pd
import plotly.io as pio

fig = go.Figure(data=[go.Table(
    header=dict(values=dff.columns,
                fill_color='#96c6e2',
                align='center'),
    cells=dict(values=[dff['Region'],dff['Test Statistic'], dff['p-value'], dff['alpha'],dff['Hypothesis']],
               fill_color='#eaeaf2',
               align='left'))
])

fig.update_layout(
    height=650,
    width = 700,
    title_text="Two Tailed T-Test Between Whole Brain and ROI Decision Scores",
    title_x=0.5
)
import os

if not os.path.exists("data"):
    os.mkdir("data")
fig.write_image("data/dscore_rois_ttst.png", scale = 2)
fig.show()